In [6]:
import pandas as pd
import json

df = pd.read_csv("/ltstorage/home/strich/STASC/logs/test_data_qwen_2.5_1.5b_hotpot_baseline_cot.csv").head(10)
df["initial_generation"][0]  # Display the first initial generation

'["Step 1: Identify the victim of Singapore\'s caning punishment.\\nThe victim is Oliver Fricker.\\n\\nStep 2: Determine the nationality of the victim Oliver Fricker.\\nOliver Fricker is British.\\n\\nStep 3: Identify the nationality of the foreign born victim of Singapore\'s caning punishment.\\nThe foreign born victim of Singapore\'s caning punishment is also British.\\n\\nFinal Answer: The nationality of the foreign born victim of Singapore\'s caning punishment is British."]'

In [7]:
with open("/ltstorage/home/strich/STASC/outputs/25-08-11_18:41_qwen_2.5_1.5b/inference_log.json", "r") as f:
    data = json.load(f)

df_json = pd.DataFrame(data)
df_json["response"][0]  # Display the first response from the JSON data

"step 1: identify the context of the question - the question is about the nationality of a foreign-born victim of singapore's caning punishment.\nstep 2: determine the nationality of the victim - the victim was oliver fricker, a british national.\nstep 3: confirm the nationality of the foreign-born victim - the foreign-born victim was also oliver fricker, a british national.\nstep 4: conclude with the final answer - the nationality of the foreign-born victim of singapore's caning punishment before oliver fricker experienced the same was british.\nfinal answer: british"

In [2]:
df

,dataset,question_id,question_text,level,type,answers_objects,contexts,reference,retrieved_contexts,context_response,...,Eccentricity_NLI_score_entail,LexicalSimilarity_rougeL,adaptive_rag_one_retrieve,adaptive_rag_no_retrieve,new_retriever_adaptive_rag_one_retrieve,new_retriever_adaptive_rag_no_retrieve,MahalanobisDistanceSeq_decoder,RelativeMahalanobisDistanceSeq_decoder,RDESeq_decoder,initial_generation
0,hotpotqa,5a77666555429966f1a36d1f,What is the nationality of the foreign born vi...,hard,bridge,"[{'date': {'day': '', 'month': '', 'year': ''}...","[{'idx': 0, 'is_supporting': False, 'paragraph...",['American'],"[""Title: Oliver Fricker. Content: Oliver Frick...",Michael P. Fay.,...,2.828427,-0.370723,American,American,Michael P. Fay,American,2639.500244,1883.850464,392.587112,"[""Step 1: Identify the victim of Singapore's c..."
1,hotpotqa,5ab6ba045542995eadef007e,Dorian Gray is the main character of what phil...,hard,bridge,"[{'date': {'day': '', 'month': '', 'year': ''}...","[{'idx': 0, 'is_supporting': False, 'paragraph...",['The Picture of Dorian Gray'],['Title: The Picture of Dorian Gray. Content: ...,The Picture of Dorian Gray.,...,2.645751,-0.468889,The Picture of Dorian Gray,Dorian Gray,The Picture of Dorian Gray,Dorian Gray,3921.310791,2943.398926,1430.203895,['Step 1: Identify the novel\'s title.\nThe qu...
2,hotpotqa,5a810221554299260e20a1f9,Who wrote the 1970 international hit song Murr...,hard,bridge,"[{'date': {'day': '', 'month': '', 'year': ''}...","[{'idx': 0, 'is_supporting': False, 'paragraph...",['Andrew Lloyd Webber and Tim Rice'],['Title: Murray Head. Content: Murray Seafield...,Andrew Lloyd Webber,...,2.236070,-0.515079,Tim Rice and Andrew Lloyd Webber,Leon Russell and Bonnie Bramlett,Andrew Lloyd Webber and Tim Rice,Leon Russell and Bonnie Bramlett,2190.012695,1271.307739,82.022718,['Step 1: Identify the 1970 international hit ...
3,hotpotqa,5ae8161555429952e35eaa04,Where was the match held where Chicharito scor...,hard,bridge,"[{'date': {'day': '', 'month': '', 'year': ''}...","[{'idx': 0, 'is_supporting': False, 'paragraph...",['Wembley Stadium'],"[""Title: Dimitar Berbatov. Content: Dimitar Iv...","Wembley Stadium, London.",...,3.000000,-0.337937,"Wembley Stadium, London",Old Trafford,Old Trafford,Old Trafford,2078.424561,1415.884277,533.306298,['Step 1: Identify the players mentioned in th...
4,hotpotqa,5ae52bc75542992663a4f12d,Where was Nexon who was composed by Adam Gubma...,hard,bridge,"[{'date': {'day': '', 'month': '', 'year': ''}...","[{'idx': 0, 'is_supporting': False, 'paragraph...",['South Korea'],['Title: Adam Gubman. Content: Adam Gubman (Ap...,There is no information about a location named...,...,3.000000,-0.308047,Adam Gubman composed music for more than 550 v...,PlayStation 2,"Seoul, South Korea",PlayStation 2,1858.785400,1058.167725,53.498359,['Step 1: Identify Nexon as a song composed by...
5,hotpotqa,5a8ba07d5542996e8ac88985,What event in 1920 was established by the dire...,hard,bridge,"[{'date': {'day': '', 'month': '', 'year': ''}...","[{'idx': 0, 'is_supporting': False, 'paragraph...",['Salzburg Festival'],['Title: Miracle. Content: A miracle is an eve...,There is no information in the passages that l...,...,3.000000,-0.285134,"The director of the play \""The Miracle\"" is Ma...",Salzburg Festival,1920,Salzburg Festival,2167.755127,1263.620850,75.622281,"['Step 1: Identify the director of the play ""T..."
6,hotpotqa,5a8d40715542994ba4e3dc3a,This annual event held in several major cities...,hard,bridge,"[{'date': {'day': '', 'month': '', 'year': ''}...","[{'idx': 0, 'is_supporting': False, 'paragraph...",['the summer solstice'],['Title: Night of the Arts. Content: The Night...,summer solstice,...,2.449490,-0.582804,the summer solstice in June,summer solstice,summer solstice,summer solstice,4280.787109,3311.678467,1712.987184,['Step 1: Identify the key elements of the que...
7,hotpotqa,5ac2f79e5542996773102661,Chun Woo-hee's notable films include a South ...,hard,bridge,"[{'d